# Record Embedding

In [33]:
import pandas as pd
import numpy as np
import os
import calendar
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

In [34]:
df = pd.read_csv('amazonCleaned.csv', dtype=object)

In [35]:
df.head()

,Name,Author,Format,Publisher,Publishing Date
0,Age of Myth: Book One of The Legends of the Fi...,Michael J. Sullivan,Paperback,Del Rey,"January 31, 2017"
1,Rise of the Dragons (Kings and Sorcerers--Book 1),Morgan Rice,Hardcover,Morgan Rice,"August 4, 2017"
2,The Book of Deacon (Volume 1),Joseph Lallo,Kindle,CreateSpace Independent Publishing Platform,"March 18, 2012"
3,A Quest of Heroes: Book #1 in the Sorcerer's Ring,Morgan Rice,Hardcover,Morgan Rice,"December 3, 2012"
4,Fantasia: An Algerian Cavalcade,Dorothy S. Blair,Kindle,Heinemann; 1 edition,"March 15, 1993"


## Embedding using fastText
Details here: https://fasttext.cc/

In [36]:
import warnings
warnings.filterwarnings('ignore')

import gensim
from gensim.models import FastText

Convert each tuple into a row

In [37]:
names = df['Name'].tolist()
authors = df['Author'].tolist()
publishers = df['Publisher'].tolist()
dates = df['Publishing Date'].tolist()
formats = df['Format'].tolist()

combined = list(zip(names, authors, publishers, dates, formats))

In [38]:
len(combined)
combined[0]

('Age of Myth: Book One of The Legends of the First Empire',
 'Michael J. Sullivan',
 'Del Rey',
 'January 31, 2017',
 'Paperback')

In [ ]:
#train fasttext - cbow
model_fast = FastText(combined, min_count=1, workers=8, iter=1000)

In [ ]:
model_fast.most_similar('Age of Myth: Book One of The Legends of the First Empire')

In [ ]:
print(model_fast)

In [ ]:
print(model_fast['Tor Fantasy'])

In [ ]:
model_fast.most_similar("Harry Potter And The Goblet Of Fire")

In [ ]:
model_fast.most_similar("Bloomsbury")

In [ ]:
model_fast.most_similar('March 3, 2015')

In [ ]:
model_fast.most_similar('The Way of Kings')

In [ ]:
model_fast.most_similar("Brandon Sanderson")

In [ ]:
model_fast.save("amazonModelFastText.w2v")

In [ ]:
#hospital dataset

#1. load hospital dataset and turthvalue.csv; mark error for unmatched values and cleanup using FastText; fill empty cells with word2vec
#finding errors and outliers -> discover FDs and try marking errors? or word2vec most_similar of val and if row does not contain the cell values??!

df_hospital = pd.read_csv('sorted_output.csv', dtype=object)
df_truth = pd.read_csv('truthvalue.csv', dtype=object)

df_hospital.head()

provider = df_truth['ProviderNumber'].tolist()
hospital = df_truth['HospitalName'].tolist()
address = df_truth['Address1'].tolist()
city = df_truth['City'].tolist()
state = df_truth['State'].tolist()
zipcode = df_truth['ZipCode'].tolist()
county = df_truth['CountyName'].tolist()
phone = df_truth['PhoneNumber'].tolist()
hospType = df_truth['HospitalType'].tolist()
owner = df_truth['HospitalOwner'].tolist()
service = df_truth['EmergencyService'].tolist()
condition = df_truth['Condition'].tolist()
code = df_truth['MeasureCode'].tolist()
name = df_truth['MeasureName'].tolist()
score = df_truth['Score'].tolist()
sample = df_truth['Sample'].tolist()
stateavg = df_truth['Stateavg'].tolist()

combined_hosp = list(zip(provider, hospital, address, city, state, zipcode, county, phone, hospType, owner, service, condition, code, name, score, sample, stateavg))

In [ ]:
#hospital dirty dataset; maintain dirty dataset separately
provider = df_hospital['provider'].tolist()
hospital = df_hospital['hospital name'].tolist()
address = df_hospital['address1'].tolist()
city = df_hospital['city'].tolist()
state = df_hospital['state'].tolist()
zipcode = df_hospital['zip code'].tolist()
county = df_hospital['county name'].tolist()
phone = df_hospital['phone number'].tolist()
hospType = df_hospital['hospital type'].tolist()
owner = df_hospital['hospital owner'].tolist()
service = df_hospital['emergency service'].tolist()
condition = df_hospital['condition'].tolist()
code = df_hospital['measure code'].tolist()
name = df_hospital['measure name'].tolist()
score = df_hospital['score'].tolist()
sample = df_hospital['sample'].tolist()
stateavg = df_hospital['stateavg'].tolist()

combined_dirty = list(zip(provider, hospital, address, city, state, zipcode, county, phone, hospType, owner, service, condition, code, name, score, sample, stateavg))
len(combined_hosp)
combined_hosp[0]

In [ ]:
#Calculating error or dirty cells
dirty_list = []

for i in range(len(combined_dirty)):
    for j in range(len(combined_dirty[0])):
        if combined_dirty[i][j] != combined_hosp[i][j]:
            dirty_list.append(combined_dirty[i][j]) 

In [ ]:
#train fasttext - cbow for hospital dataset
model_hosp = FastText(combined_hosp, min_count=1, workers=8, iter=100)

#compare df_hospital and combined[i]. Mark error for any difference in values by maintaining them in separate list. 
#Perform model_hosp.most_similar(error_cell) and update the error value with closest similarity measure
#Evaluate accuracy with df_truth
#next step - determine errors or outliers
for i in range(len(dirty_list)):
    for j in range(len(dirty_list[0])):
        model_hosp.most_similar(dirty_list[i][j])
        #TODO: Take the top possible values or top value (for now) and correct the error.
        #Once done calculate the diff/accuracy with df_truth

In [ ]:
model_hosp.save("hospitalModelFastText.w2v")